<a href="https://colab.research.google.com/github/HakaiInstitute/hakai-profile-qaqc/blob/development/notebooks/review_remote_drops.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Hakai API client

In [1]:
%pip install hakai_api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2022.7.1
    Uninstalling pytz-2022.7.1:
      Successfully uninstalled pytz-2022.7.1
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling requests-2.27.1:
      Successfully uninstalled requests-2.27.1


## Import Packages

In [8]:
from hakai_api import Client
import pandas as pd
import folium

## Get Credentials From Hakai
This step creates an Hakai API connection, retrieve your Hakai account information and creates a temporary token saved locally in your temporary files to be able to recognize you and give you access to Hakai's data.

In [4]:
client = Client()

Please go here and authorize:
https://hecate.hakai.org/api-client-login

Copy and past your credentials from the login page:
token_type=Bearer&access_token=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpZCI6IjExNjMxNzQzNDYzMTA5NzUwOTczOSIsImVtYWlsIjoiamVzc3kuYmFycmV0dGVAaGFrYWkub3JnIiwidmVyaWZpZWRfZW1haWwiOnRydWUsIm5hbWUiOiJKZXNzeSBCYXJyZXR0ZSIsImdpdmVuX25hbWUiOiJKZXNzeSIsImZhbWlseV9uYW1lIjoiQmFycmV0dGUiLCJsaW5rIjoiaHR0cHM6Ly9wbHVzLmdvb2dsZS5jb20vMTE2MzE3NDM0NjMxMDk3NTA5NzM5IiwicGljdHVyZSI6Imh0dHBzOi8vbGgzLmdvb2dsZXVzZXJjb250ZW50LmNvbS9hL0FHTm15eFlDTkEyVDl1ZUZWMDJpeTJ0am9RM2tGNmRUanVkSG1hcHlVWS15PXM5Ni1jIiwibG9jYWxlIjoiZW4iLCJoZCI6Imhha2FpLm9yZyIsImdyb3VwcyI6WyJqZXNzeS5iYXJyZXR0ZUBoYWthaS5vcmciLCJhbGdhZWV4cGxvcmVyQGhha2FpLm9yZyIsImNpb29zLnBhY2lmaWNAaGFrYWkub3JnIiwiZGF0YS5tb2JpbGl6YXRpb25AaGFrYWkub3JnIiwiZWltcy5hdXRoLmhha2FpQGhha2FpLm9yZyIsImVpbXMuYXV0aC5uYXR1cmUudHJ1c3RAaGFrYWkub3JnIiwiZWltcy5hdXRoLnBhcmtzLmNhbmFkYUBoYWthaS5vcmciLCJlaW1zLmF1dGguc2ZjQGhha2FpLm9yZyIsImVpbXMuYXV0aC51d3RAaGFrYWk

## Download Data
This section download some data from the Hakai database, through the Hakai API. For a more information about the Hakai API, see https://hakaiinstitute.github.io/hakai-api/

In [67]:
# This endpoint retrieve the hakai ctd data 
# (see here for more endpoints https://hakaiinstitute.github.io/hakai-api/endpoints/)
endpoint = "ctd/views/file/cast" 

# This section of the url will filter the ctd data on the database 
# and provide use only the data matchingg those conditions:
# (see documentation here https://hakaiinstitute.github.io/hakai-api/querying_data/)
filter_endpoint = (
    "location_flag_level_1={3,4}" # at station QU39
    + "&limit=-1" # Get all the data that respects those conditions above
)

# Generate the query and send query to Hakai API
response = client.get(f"{client.api_root}/{endpoint}?{filter_endpoint}")
response.raise_for_status()
df = pd.DataFrame(response.json())
print("loaded %s drops" % len(df))



loaded 501 drops


## Generate Derived Variables
We can use the TEOS-10 standard python package (GSW Oceanographic Toolbox) to generate oceanographic variables from different measurement made by the CTD instruments.

In [32]:
len(df.query("location_flag_level_1 ==3"))

202

## Plot Data
The graphic library **Plotly** (https://plotly.com/python/) is a great tool to generate nice interactive figures with just a few lines of code. 

The plotly express package is certainly the easiest to use: https://plotly.com/python/plotly-express/

## Hakai bad locations

In [65]:
def _get_popup(drop):
    return "\n".join([f"{key}: {value}" for key,value in drop[['station','hakai_id']].items()])
m = folium.Map(location=df[['latitude','longitude']].mean(),zoom_start=7)
for id, row in df.query('work_area not in ("ESTUARY")').iterrows():
    folium.CircleMarker(radius=1,location=row[['station_latitude','station_longitude']].values, color="crimson",fill=True).add_to(m)
    folium.PolyLine([row[['latitude','longitude']].values, row[['station_latitude','station_longitude']].values], popup=_get_popup(row), opacity=0.5).add_to(m)
m.save('hakai-bad-locations.html')
m

# Nature Trust bad locations

In [66]:
def _get_popup(drop):
    return "\n".join([f"{key}: {value}" for key,value in drop[['station','hakai_id']].items()])
m = folium.Map(location=df[['latitude','longitude']].mean(),zoom_start=5)
for id, row in df.query('work_area in ("ESTUARY")').iterrows():
    folium.CircleMarker(radius=1,location=row[['station_latitude','station_longitude']].values, color="crimson",fill=True).add_to(m)
    folium.PolyLine([row[['latitude','longitude']].values, row[['station_latitude','station_longitude']].values], popup=_get_popup(row), opacity=0.5).add_to(m)
m.save('nt-bad-locations.html')
m